In [81]:
import os
import pandas as pd
from corrLib import readdata
from utils import *
import numpy as np

## See the current status

Run the cell below to know what has or has not been done!

In [163]:
data_folder = '/home/zhengyang/data/DE/'
log = pd.read_csv(os.path.join(data_folder, 'main_log.csv'), header=[0,1])
log.head() # to see the whole table, comment out the .head()

params                                                               \
      Date                                     Subfolder  BC  MPP   FPS   
0  7132021                          2021-07-13_17h17m24s NaN  0.1  30.0   
1  7202021  1st-drop-tracking/2021-07-20_16h24m06s-mttry NaN  0.1  30.0   
2  7202021     1st-drop-tracking/2021-07-20_16h25m35s-m2 NaN  0.1  30.0   
3  7202021     1st-drop-tracking/2021-07-20_16h27m01s-m3 NaN  0.1  30.0   
4  7202021     1st-drop-tracking/2021-07-20_16h28m21s-m4 NaN  0.1  30.0   

  analysis                                      Visual                          
        ID     OD 2D-trajectory 3D-trajectory 2D-image 3D-projection 3D-outRef  
0     34.5  100.5           1.0           1.0      1.0           0.0       0.0  
1      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
2      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
3      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
4      NaN    NaN           1.0           0.0      0.0           0.0       0.0

## Update the log

- frame rate from RawImageInfo.txt
- check the dropSize.xlsx for ID and OD
- check traj.csv and xyz-traj.csv for 2D and 3D trajectory
- check key visualization files
    - 2D-image: "2d-traj.jpg"
    - 3D-projection: "3d-traj-animation.avi"
    - 3D-outRef: "3d-traj-RVRef.avi"

In [158]:
for num, i in log.iterrows():
    sf = os.path.join(data_folder, '{:08d}'.format(i[('params', 'Date')]), i[('params', 'Subfolder')])
    # RawImageInfo.txt
    imageInfo_dir = os.path.join(sf, 'RawImageInfo.txt')
    if os.path.exists(imageInfo_dir) == True:
        with open(imageInfo_dir, 'r') as f:
            fps = int(f.readline())
        log[('params', 'FPS')].at[num] = fps

    # dropSize.xlsx
    dropSize_dir = os.path.join(sf, 'dropSize.xlsx')
    if os.path.exists(dropSize_dir) == True:
        outer, inner = read_dropSize(dropSize_dir)
        log[('analysis', 'OD')].at[num] = outer
        log[('analysis', 'ID')].at[num] = inner
    
    # traj.csv
    traj_dir = os.path.join(sf, 'crop_HoughCircles', 'traj.csv')
    log[('analysis', '2D-trajectory')].at[num] = int(os.path.exists(traj_dir))
    # xyz-traj.csv
    xyz_dir = os.path.join(sf, 'crop_HoughCircles', 'xyz-traj.csv')
    log[('analysis', '3D-trajectory')].at[num] = int(os.path.exists(xyz_dir))
    # 2d-image.jpg
    image_2D_dir = os.path.join(sf, 'crop_HoughCircles', '2d-traj.jpg')
    log[('Visual', '2D-image')].at[num] = int(os.path.exists(image_2D_dir))
    # 3d-traj-animation.avi
    projection_3D_dir = os.path.join(sf, 'crop_HoughCircles', '3d-traj-animation.avi')
    log[('Visual', '3D-projection')].at[num] = int(os.path.exists(projection_3D_dir))
    # 3d-traj-RVRef.avi
    outRef_3D_dir = os.path.join(sf, 'crop_HoughCircles', '3d-traj-RVRef.avi')
    log[('Visual', '3D-outRef')].at[num] = int(os.path.exists(outRef_3D_dir))

In [160]:
log.head()

params                                                               \
      Date                                     Subfolder  BC  MPP   FPS   
0  7132021                          2021-07-13_17h17m24s NaN  0.1  30.0   
1  7202021  1st-drop-tracking/2021-07-20_16h24m06s-mttry NaN  0.1  30.0   
2  7202021     1st-drop-tracking/2021-07-20_16h25m35s-m2 NaN  0.1  30.0   
3  7202021     1st-drop-tracking/2021-07-20_16h27m01s-m3 NaN  0.1  30.0   
4  7202021     1st-drop-tracking/2021-07-20_16h28m21s-m4 NaN  0.1  30.0   

  analysis                                      Visual                          
        ID     OD 2D-trajectory 3D-trajectory 2D-image 3D-projection 3D-outRef  
0     34.5  100.5           1.0           1.0      1.0           0.0       0.0  
1      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
2      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
3      NaN    NaN           1.0           0.0      0.0           0.0       0.0  
4      NaN    NaN           1.0           0.0      0.0           0.0       0.0

## Save the log

In [162]:
log.to_csv(os.path.join(data_folder, 'main_log.csv'), index=False)